In [1]:
import pandas as pd
import datetime as dt
import dash
from dash import dcc, html, Input, Output
import tab1
import tab2
import tab3
import os
import plotly.graph_objects as go

In [2]:
class db:
    def __init__(self):
        self.transactions = db.transactions_init()
        self.cc = pd.read_csv(r'db\country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')
    @staticmethod
    def transactions_init():
        transactions = pd.DataFrame()
        src = r'db\transactions'
        dfs = []
        for filename in os.listdir(src):
            path = os.path.join(src, filename)
            df = pd.read_csv(path, index_col=0)
            print(f"{filename}: {df.columns.tolist()}")
            dfs.append(df)
        transactions = pd.concat(dfs)

        print("Połączono:", transactions.shape)

        if 'tran_date' not in transactions.columns:
            raise ValueError("'tran_date' column is missing!")

        def convert_dates(x):
            try:
                return dt.datetime.strptime(x,'%d-%m-%Y')
            except:
                return dt.datetime.strptime(x,'%d/%m/%Y')
                
        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))
        return transactions
    def merge(self):
        df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code']).set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')
        df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code']).set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')
        df = df.join(self.customers.join(self.cc,on='country_code').set_index('customer_Id'),on='cust_id')
        self.merged = df

In [3]:
df = db()
df.merge()

transactions-2016.csv: ['transaction_id', 'cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt', 'Store_type']
transactions-2017.csv: ['transaction_id', 'cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt', 'Store_type']
transactions-2018.csv: ['transaction_id', 'cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt', 'Store_type']
transactions-2019.csv: ['transaction_id', 'cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt', 'Store_type']
Połączono: (23053, 10)


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

In [5]:
app.layout = html.Div([html.Div([dcc.Tabs(id='tabs',value='tab-1',children=[
                            dcc.Tab(label='Sprzedaż globalna',value='tab-1'),
                            dcc.Tab(label='Produkty',value='tab-2'),
                            dcc.Tab(label='Kanały sprzedaży', value='tab-3')
                            ]),
                            html.Div(id='tabs-content')
                    ],style={'width':'80%','margin':'auto'})],
                    style={'height':'100%'})

In [6]:
@app.callback(Output('tabs-content','children'),[Input('tabs','value')])
def render_content(tab):

    if tab == 'tab-1':
        return tab1.render_tab(df.merged)
    elif tab == 'tab-2':
        return tab2.render_tab(df.merged)
    elif tab == 'tab-3':
        return tab3.render_tab(df.merged)

In [7]:
## tab1 callbacks
@app.callback(Output('bar-sales','figure'),
    [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_bar_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby([pd.Grouper(key='tran_date',freq='M'),'Store_type'])['total_amt'].sum().round(2).unstack()

    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(x=grouped.index,y=grouped[col],name=col,hoverinfo='text',
        hovertext=[f'{y/1e3:.2f}k' for y in grouped[col].values]))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(title='Przychody',barmode='stack',legend=dict(x=0,y=-0.5)))

    return fig

In [8]:
@app.callback(Output('choropleth-sales','figure'),
            [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_choropleth_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby('country')['total_amt'].sum().round(2)

    trace0 = go.Choropleth(colorscale='Viridis',reversescale=True,
                            locations=grouped.index,locationmode='country names',
                            z = grouped.values, colorbar=dict(title='Sales'))
    data = [trace0]
    fig = go.Figure(data=data,layout=go.Layout(title='Mapa',geo=dict(showframe=False,projection={'type':'natural earth'})))

    return fig

In [9]:
## tab2 callbacks
@app.callback(Output('barh-prod-subcat','figure'),
            [Input('prod_dropdown','value')])
def tab2_barh_prod_subcat(chosen_cat):

    grouped = df.merged[(df.merged['total_amt']>0)&(df.merged['prod_cat']==chosen_cat)].pivot_table(index='prod_subcat',columns='Gender',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['F']+x['M']).sort_values(by='_sum').round(2)

    traces = []
    for col in ['F','M']:
        traces.append(go.Bar(x=grouped[col],y=grouped.index,orientation='h',name=col))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(barmode='stack',margin={'t':20,}))
    return fig

In [10]:
## tab3 callbacks
@app.callback(Output('weekdays-sales', 'figure'), [Input('tabs', 'value')])
def tab3_weekday_sales(tab_value):
    df_filtered = df.merged[df.merged['total_amt'] > 0].copy()
    df_filtered['weekday'] = df_filtered['tran_date'].dt.dayofweek
    order = ['Pon', 'Wt', 'Śr', 'Czw', 'Pt', 'Sob', 'Niedz']
    df_filtered['weekday'] = df_filtered['weekday'].map(dict(enumerate(order)))

    grouped = df_filtered.groupby(['weekday', 'Store_type'])['total_amt'].sum().unstack().reindex(order)

    traces = []
    for col in grouped.columns:
        traces.append(go.Scatter(x=grouped.index, y=grouped[col], mode='lines+markers', name=col))

    fig = go.Figure(data=traces, layout=go.Layout(title='Sprzedaż wg dnia tygodnia', xaxis_title='Dzień tygodnia', yaxis_title='Sprzedaż'))

    return fig

In [11]:
@app.callback(Output('customer_age','figure'),
            [Input('tabs','value')])
def tab3_customer_age(tab_value):
    df.merged['age'] = (pd.to_datetime('today') - pd.to_datetime(df.merged['DOB'])).dt.days // 365
    
    bins = [0, 29, 44, 59, 120]
    labels = ['18-29', '30-44', '45-59', '60+']
    df.merged['age_group'] = pd.cut(df.merged['age'], bins=bins, labels=labels, right=True)

    grouped = df.merged[df.merged['total_amt'] > 0].groupby(['age_group', 'Store_type'])['total_amt'].sum().unstack().fillna(0).round(2)
    
    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(name=col, x=grouped.index, y=grouped[col]))
    
    fig = go.Figure(data=traces)
    fig.update_layout(
        barmode='group',
        title='Udział grup wiekowych w sprzedaży wg kanału',
        xaxis_title='Grupa wiekowa',
        yaxis_title='Wartość sprzedaży'
    )
    return fig

In [12]:
df.merged.head(2)

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,prod_cat,prod_subcat,DOB,Gender,country_code,country
16062,40303022895,272142,2016-12-31,1,2,5,537,281.925,2966.925,Flagship store,Footwear,Women,13-10-1988,M,8.0,Denmark
16063,47557596721,273764,2016-12-31,8,3,-1,-1037,108.885,-1145.885,Flagship store,Electronics,Personal Appliances,13-03-1982,M,8.0,Denmark


In [13]:
if __name__ == '__main__':
    app.run(debug=True, port=8055)